In [ ]:
!pip install -U sentence-transformers
!pip install transformers
!pip install datasets

In [ ]:
import json
import heapq
import os
import re
import shutil
import warnings

from datasets import load_dataset, load_metric
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
tokenizer_codegen = AutoTokenizer.from_pretrained('Salesforce/codegen-350M-mono', pad_token='<pad>')
model_codegen = AutoModelForCausalLM.from_pretrained('Salesforce/codegen-350M-mono', pad_token_id=50256)

In [ ]:
NUM_SAMPLES = 0
os.environ["HF_ALLOW_CODE_EVAL"] = "1" # for code_eval

In [ ]:
dataset = load_dataset('openai_humaneval', split='test')

if NUM_SAMPLES > 0 and NUM_SAMPLES < len(dataset):
  subset = dataset.select(range(NUM_SAMPLES))
else:
  subset = dataset

print(len(subset))

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

Dataset openai_humaneval downloaded and prepared to /root/.cache/huggingface/datasets/openai_humaneval/openai_humaneval/1.0.0/2955cebd73602e828fa8c0a424c594e5fab4ec863b316ca98f3d8fdb6a626e75. Subsequent calls will reuse this data.
164


In [ ]:
# build whitelist and bad_words_ids
text = ''
for idx in range(len(subset)):
  text += subset[idx]['canonical_solution']

symbols = ['(', ')', '[', ']', '{', '}', ',', '.', ';', ':', '+', '-', '*', '/', '%', '<', '>', '=', '!', '&', '|', '^', '~', '@', '#', '$', '?']
for symbol in symbols:
  text = text.replace(symbol, ' ')

words = text.split()
words_set = set(words)
whitelist = symbols + list(words_set)

whitelist_ids = tokenizer_codegen(whitelist).input_ids
whitelist_ids = [item for sublist in whitelist_ids for item in sublist]
whitelist_ids = list(set(whitelist_ids))

vocab = tokenizer_codegen.get_vocab()
vocab_ids = list(vocab.values())

def create_bad_words_ids(vocab_ids, whitelist_ids):
  bad_word_ids = vocab_ids
  for idx in range(len(whitelist_ids)):
    if whitelist_ids[idx] in bad_word_ids:
      bad_word_ids.remove(whitelist_ids[idx])
  return bad_word_ids

bad_word_ids = create_bad_words_ids(vocab_ids, whitelist_ids)
bad_word_ids

[18226,
 40048,
 20207,
 36241,
 19427,
 3296,
 22168,
 39694,
 47651,
 48481,
 10709,
 24725,
 44525,
 32834,
 37868,
 12849,
 9402,
 3946,
 29505,
 43561,
 45288,
 15321,
 27890,
 27578,
 36825,
 39425,
 45811,
 36103,
 40292,
 8628,
 3080,
 6198,
 31764,
 43424,
 48640,
 35546,
 22554,
 40665,
 49769,
 46154,
 49856,
 9172,
 37697,
 25958,
 19694,
 41243,
 35418,
 42858,
 32213,
 40736,
 49437,
 26454,
 5526,
 7828,
 27797,
 29508,
 1235,
 47037,
 30370,
 21827,
 13470,
 24523,
 13107,
 19481,
 18624,
 37085,
 12871,
 997,
 7121,
 7710,
 9735,
 20912,
 30728,
 28988,
 34254,
 39548,
 41531,
 22106,
 39811,
 11835,
 48838,
 16469,
 7246,
 43369,
 41708,
 1980,
 18436,
 30640,
 3039,
 36868,
 32033,
 1843,
 39869,
 44881,
 45749,
 8535,
 25270,
 49557,
 1738,
 6360,
 17656,
 32295,
 13566,
 24654,
 23035,
 43473,
 20423,
 45920,
 804,
 6837,
 23586,
 33715,
 23912,
 1506,
 2712,
 7917,
 19242,
 4521,
 39916,
 179,
 19335,
 19558,
 35493,
 1590,
 7781,
 27092,
 8171,
 36654,
 40884,
 1

In [ ]:
with open('bad_word_ids.txt', 'w') as f:
  for item in bad_word_ids:
    f.write("%s\n" % item)

shutil.copyfile('/content/bad_word_ids.txt', '/content/drive/MyDrive/UZH/AI4PP/bad_word_ids.txt')

'/content/drive/MyDrive/UZH/AI4PP/bad_word_ids.txt'

In [ ]:
with open('/content/drive/MyDrive/UZH/AI4PP/bad_word_ids.txt', 'r') as f:
  bad_word_ids = f.read().splitlines()

for idx in range(len(bad_word_ids)):
  bad_word_ids[idx] = int(bad_word_ids[idx])
bad_word_ids = [bad_word_ids]
print(bad_word_ids)

[[18226, 40048, 20207, 36241, 19427, 3296, 22168, 39694, 47651, 48481, 10709, 24725, 44525, 32834, 37868, 12849, 9402, 3946, 29505, 43561, 45288, 15321, 27890, 27578, 36825, 39425, 45811, 36103, 40292, 8628, 3080, 6198, 31764, 43424, 48640, 35546, 22554, 40665, 49769, 46154, 49856, 9172, 37697, 25958, 19694, 41243, 35418, 42858, 32213, 40736, 49437, 26454, 5526, 7828, 27797, 29508, 1235, 47037, 30370, 21827, 13470, 24523, 13107, 19481, 18624, 37085, 12871, 997, 7121, 7710, 9735, 20912, 30728, 28988, 34254, 39548, 41531, 22106, 39811, 11835, 48838, 16469, 7246, 43369, 41708, 1980, 18436, 30640, 3039, 36868, 32033, 1843, 39869, 44881, 45749, 8535, 25270, 49557, 1738, 6360, 17656, 32295, 13566, 24654, 23035, 43473, 20423, 45920, 804, 6837, 23586, 33715, 23912, 1506, 2712, 7917, 19242, 4521, 39916, 179, 19335, 19558, 35493, 1590, 7781, 27092, 8171, 36654, 40884, 13811, 42871, 7213, 26445, 43734, 45747, 7243, 1924, 42836, 20651, 34346, 49705, 50109, 262, 390, 17867, 26185, 29622, 38827, 230

In [ ]:
code_eval_metric = load_metric("code_eval")

<ipython-input-9-4b305cb29287>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  code_eval_metric = load_metric("code_eval")


In [ ]:
def evaluation(predictions, metric):
  references = []

  for idx in range(len(predictions)):
    test_func = subset[idx]["test"]
    entry_point = f"check({subset[idx]['entry_point']})"
    references.append("\n" + test_func + "\n" + entry_point)

  pass_at_k, results = metric.compute(predictions=predictions, references=references)

  return pass_at_k, results

In [ ]:
def evaluate(dataset, in_context=False, whitelist=False, num_test=0):
  # normal completion
  predictions = []

  if num_test<1 or num_test>len(dataset):
    num_test = len(dataset)

  for idx in range(num_test):
    if in_context:
      text = generate_prompt(idx, dataset)
    else:
      text = dataset[idx]['prompt']
    
    input_ids = tokenizer_codegen(text, return_tensors="pt").input_ids

    if whitelist:
      generated_ids = model_codegen.generate(input_ids, max_new_tokens=128, bad_words_ids=bad_word_ids)
    else:
      generated_ids = model_codegen.generate(input_ids, max_new_tokens=128)

    prediction = tokenizer_codegen.decode(generated_ids[0], skip_special_tokens=True)

    # remove unfinished block
    last_double_newline = re.search(r'.*\n\n\n', prediction[::-1])
    if(last_double_newline!=None):
      last_double_newline_index = len(prediction) - last_double_newline.end()
      prediction = prediction[:last_double_newline_index]

    predictions.append([prediction])

  pass_at_k, results = evaluation(predictions, code_eval_metric)

  return pass_at_k, results

In [ ]:
# test
pass_at_k, results = evaluate(subset, whitelist=True, num_test=10)
print(pass_at_k)
print(results)
pass_at_k, results = evaluate(subset, whitelist=False, num_test=10)
print(pass_at_k)
print(results)

{'pass@1': 0.2}
defaultdict(<class 'list'>, {1: [(0, {'task_id': 1, 'passed': False, 'result': 'failed: ', 'completion_id': 0})], 0: [(0, {'task_id': 0, 'passed': True, 'result': 'passed', 'completion_id': 0})], 2: [(0, {'task_id': 2, 'passed': False, 'result': 'failed: ', 'completion_id': 0})], 3: [(0, {'task_id': 3, 'passed': False, 'result': 'failed: list index out of range', 'completion_id': 0})], 4: [(0, {'task_id': 4, 'passed': False, 'result': 'failed: ', 'completion_id': 0})], 5: [(0, {'task_id': 5, 'passed': False, 'result': 'failed: ', 'completion_id': 0})], 9: [(0, {'task_id': 9, 'passed': False, 'result': 'failed: ', 'completion_id': 0})], 6: [(0, {'task_id': 6, 'passed': False, 'result': "failed: invalid literal for int() with base 10: ')'", 'completion_id': 0})], 7: [(0, {'task_id': 7, 'passed': True, 'result': 'passed', 'completion_id': 0})], 8: [(0, {'task_id': 8, 'passed': False, 'result': 'failed: ', 'completion_id': 0})]})
{'pass@1': 0.2}
defaultdict(<class 'list'>, 

In [ ]:
# full dataset test
pass_at_k, results = evaluate(subset, whitelist=True)
print(pass_at_k)
print(results)
pass_at_k, results = evaluate(subset, whitelist=False)
print(pass_at_k)
print(results)

{'pass@1': 0.04878048780487805}
defaultdict(<class 'list'>, {1: [(0, {'task_id': 1, 'passed': False, 'result': 'failed: ', 'completion_id': 0})], 0: [(0, {'task_id': 0, 'passed': True, 'result': 'passed', 'completion_id': 0})], 3: [(0, {'task_id': 3, 'passed': False, 'result': 'failed: list index out of range', 'completion_id': 0})], 2: [(0, {'task_id': 2, 'passed': False, 'result': 'failed: ', 'completion_id': 0})], 5: [(0, {'task_id': 5, 'passed': False, 'result': 'failed: ', 'completion_id': 0})], 4: [(0, {'task_id': 4, 'passed': False, 'result': 'failed: ', 'completion_id': 0})], 6: [(0, {'task_id': 6, 'passed': False, 'result': "failed: invalid literal for int() with base 10: ')'", 'completion_id': 0})], 7: [(0, {'task_id': 7, 'passed': True, 'result': 'passed', 'completion_id': 0})], 9: [(0, {'task_id': 9, 'passed': False, 'result': 'failed: ', 'completion_id': 0})], 8: [(0, {'task_id': 8, 'passed': False, 'result': 'failed: ', 'completion_id': 0})], 10: [(0, {'task_id': 10, 'pas